In [4]:
import numpy as np
import os
from PIL import Image
from tqdm import tqdm
import random 
import xml.etree.ElementTree as ET

In [5]:
# check od dataset 
def del_notmatch_data(od_dataset_root, data_struct={"image": "images", "label": "labels"}, delete_=False):
    img_dir = os.path.join(od_dataset_root, data_struct["image"])
    lbl_dir = os.path.join(od_dataset_root, data_struct["label"])
    img_name_list = [f.split('.')[0] for f in os.listdir(img_dir) if f.endswith('.jpg')]
    lbl_name_list = [f.split('.')[0] for f in os.listdir(lbl_dir) if f.endswith('.xml')]
    
    img_no_label_list = [f for f in img_name_list if f not in lbl_name_list]
    print(f"{img_no_label_list=}")
    label_no_img_list = [f for f in lbl_name_list if f not in img_name_list]
    print(f"{label_no_img_list=}")
    
    if delete_:
        # delet
        for t in img_no_label_list:
            im_path = os.path.join(img_dir, t+'.jpg')
            if os.path.isfile(im_path):
                # print('can delete jpg file')
                print(f"delete {os.path.basename(im_path)}")
                os.remove(im_path)
        
        for t in label_no_img_list:
            xml_path = os.path.join(lbl_dir, t+'.xml')
            if os.path.isfile(xml_path):
                # print('can delete xml file')
                print(f"delete {os.path.basename(xml_path)}")
                os.remove(xml_path)
    
dataset_ = r"D:\ddesktop\monitoring\datadata\belthelmet\1104"
del_notmatch_data(dataset_)

img_no_label_list=[]
label_no_img_list=[]


In [9]:
# d_root = r"D:\ddesktop\monitoring\codespace\yolov8_BeltHelmet\VOCdevkit\VOC2007"
# del_notmatch_data(d_root, data_struct={"image": "JPEGImages", "label": "Annotations"}, delete_=False)
# # ok match

img_no_label_list=[]
label_no_img_list=[]


In [6]:
# del empty data
def del_empty_xml_file_and_image(image_dir, xml_dir):
    """
    删除 xml_dir 中没有任何 bbox 的 XML 文件，以及对应的图像文件。

    参数：
        image_dir (str): 存放图像文件（JPG）的目录路径。
        xml_dir (str): 存放 XML 标注文件的目录路径。
    """
    deleted_xml_files = []
    deleted_image_files = []
    total_xml_files = 0

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            total_xml_files += 1
            xml_path = os.path.join(xml_dir, xml_file)
            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
                # 查找所有的 object 元素
                objects = root.findall('object')
                if not objects:
                    # 如果没有找到 object 元素，删除该 XML 文件和对应的图像文件
                    os.remove(xml_path)
                    deleted_xml_files.append(xml_file)
                    print(f"已删除空的 XML 文件：{xml_file}")

                    # 对应的图像文件名
                    base_name = os.path.splitext(xml_file)[0]
                    # 尝试匹配不同扩展名的图像文件
                    image_extensions = ['.jpg', '.jpeg', '.png']
                    image_deleted = False
                    for ext in image_extensions:
                        image_file = base_name + ext
                        image_path = os.path.join(image_dir, image_file)
                        if os.path.exists(image_path):
                            os.remove(image_path)
                            deleted_image_files.append(image_file)
                            print(f"已删除对应的图像文件：{image_file}")
                            image_deleted = True
                            break
                    if not image_deleted:
                        print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")
            except ET.ParseError:
                # 如果 XML 文件解析错误，也删除文件和对应的图像文件
                os.remove(xml_path)
                deleted_xml_files.append(xml_file)
                print(f"已删除无法解析的 XML 文件：{xml_file}")

                # 同样删除对应的图像文件
                base_name = os.path.splitext(xml_file)[0]
                image_extensions = ['.jpg', '.jpeg', '.png']
                image_deleted = False
                for ext in image_extensions:
                    image_file = base_name + ext
                    image_path = os.path.join(image_dir, image_file)
                    if os.path.exists(image_path):
                        os.remove(image_path)
                        deleted_image_files.append(image_file)
                        print(f"已删除对应的图像文件：{image_file}")
                        image_deleted = True
                        break
                if not image_deleted:
                    print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")

    print(f"总共扫描了 {total_xml_files} 个 XML 文件，删除了 {len(deleted_xml_files)} 个空的 XML 文件。")
    print(f"删除了 {len(deleted_image_files)} 个对应的图像文件。")

img_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\images"
xml_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\labels"
del_empty_xml_file_and_image(img_d, xml_d)


总共扫描了 285 个 XML 文件，删除了 0 个空的 XML 文件。
删除了 0 个对应的图像文件。


In [7]:
# od dataset crop

# -> list: x1,y1,x2,y2
def read_bbox(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    bboxes = []
    for obj in root.findall('object'):
        
        # # 获取目标类别名称，如果需要，可以根据类别名称进行过滤
        # name = obj.find('name').text
        # if name == "drone":
        
        bbox = obj.find('bndbox')
        xmin = int(float(bbox.find('xmin').text))
        ymin = int(float(bbox.find('ymin').text))
        xmax = int(float(bbox.find('xmax').text))
        ymax = int(float(bbox.find('ymax').text))
        bboxes.append((xmin, ymin, xmax, ymax))
    return bboxes
        
def get_squared_crop_box_random(xmin, xmax, ymin, ymax, img_w, img_h, min_wsize=960):
    center_x = (xmin + xmax) // 2
    center_y = (ymin + ymax) // 2
    bbox_w = xmax - xmin
    bbox_h = ymax - ymin
    bbox_size = max(bbox_w, bbox_h)
    
    # get crop size  (crop_size, crop_size)
    min_crop_size = max(bbox_size, min_wsize)
    max_crop_size = min(img_w, img_h)
    crop_size = random.randint(min_crop_size, max_crop_size)
    
    x1 = center_x - crop_size // 2
    y1 = center_y - crop_size // 2
    x1 = max(0, min(x1, img_w - crop_size))
    y1 = max(0, min(y1, img_h - crop_size))
    x2 = x1 + crop_size
    y2 = y1 + crop_size
    return x1, x2, y1, y2


def get_16w9h_crop_box_random(xmin, xmax, ymin, ymax, img_w, img_h, min_wsize=1080, wdh=16/9, iou_th=0.6):
    """
    生成一个随机裁剪框，宽高比为16:9，包含目标对象，
    且裁剪区域与目标对象的交叉面积至少为 iou_th * bbox_area。

    参数：
        xmin, xmax, ymin, ymax: 目标对象的边界框坐标。
        img_w, img_h: 图像的宽度和高度。
        min_wsize: 裁剪框的最小宽度（默认值为1080）。
        wdh: 宽高比（默认值为16/9）。
        iou_th: 裁剪区域与目标对象的最小交叉面积比例。

    返回：
        x1, x2, y1, y2: 裁剪框的左上角和右下角坐标。
    """

    bbox_w = xmax - xmin
    bbox_h = ymax - ymin
    bbox_area = bbox_w * bbox_h

    # 计算裁剪框的最小宽度和高度，确保包含目标对象并满足最小尺寸
    crop_w_min = max(bbox_w, min_wsize)
    crop_h_min = crop_w_min / wdh

    # 确保裁剪框尺寸不超过图像尺寸
    crop_w_max = min(img_w, img_h * wdh)
    crop_h_max = crop_w_max / wdh

    max_attempts = 100  # 最大尝试次数
    for _ in range(max_attempts):
        # 在最小和最大尺寸之间随机选择裁剪宽度
        crop_w = random.uniform(crop_w_min, crop_w_max)
        crop_h = crop_w / wdh

        # 计算裁剪框的左上角坐标范围，确保包含目标对象且在图像范围内
        x1_min = max(0, xmin - (crop_w - bbox_w))
        x1_max = min(xmin, img_w - crop_w)
        y1_min = max(0, ymin - (crop_h - bbox_h))
        y1_max = min(ymin, img_h - crop_h)

        if x1_min > x1_max or y1_min > y1_max:
            continue  # 无法找到合适的裁剪框，继续下一次尝试

        # 在计算得到的范围内随机选择左上角坐标
        x1 = random.uniform(x1_min, x1_max)
        y1 = random.uniform(y1_min, y1_max)
        x2 = x1 + crop_w
        y2 = y1 + crop_h

        # 计算裁剪框与目标对象的交叉面积
        inter_x1 = max(x1, xmin)
        inter_y1 = max(y1, ymin)
        inter_x2 = min(x2, xmax)
        inter_y2 = min(y2, ymax)

        if inter_x1 < inter_x2 and inter_y1 < inter_y2:
            inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
            if inter_area >= iou_th * bbox_area:
                # 返回整数坐标
                return int(x1), int(x2), int(y1), int(y2)

    # 如果未找到合适的裁剪框，返回原始边界框
    return int(xmin), int(xmax), int(ymin), int(ymax)

def cal_iou(x1, y1, x2, y2, ax1, ay1, ax2, ay2):
    """
    计算裁剪框与目标边界框的交叉面积比例。

    参数：
        x1, y1, x2, y2: 裁剪框的坐标。
        ax1, ay1, ax2, ay2: 目标边界框的坐标。

    返回：
        iou: 交叉面积与目标边界框面积的比例。
    """
    bbox_area = (ax2 - ax1) * (ay2 - ay1)
    inter_x1 = max(x1, ax1)
    inter_y1 = max(y1, ay1)
    inter_x2 = min(x2, ax2)
    inter_y2 = min(y2, ay2)

    if inter_x1 < inter_x2 and inter_y1 < inter_y2:
        inter_area = (inter_x2 - inter_x1) * (inter_y2 - inter_y1)
        iou_ = inter_area / bbox_area
        return iou_
    else:
        return 0.0

def save_cropped_xml(original_xml_path, save_path, crop_box, new_image_size, new_image_name):
    tree = ET.parse(original_xml_path)
    root = tree.getroot()
    root.find('filename').text = new_image_name
    size = root.find('size')
    size.find('width').text = str(new_image_size[0])
    size.find('height').text = str(new_image_size[1])
    x1_crop, x2_crop, y1_crop, y2_crop = crop_box
    objects = root.findall('object')
    for obj in objects:
        bbox = obj.find('bndbox')
        xmin = int(float(bbox.find('xmin').text))
        ymin = int(float(bbox.find('ymin').text))
        xmax = int(float(bbox.find('xmax').text))
        ymax = int(float(bbox.find('ymax').text))
        xmin_new = xmin - x1_crop
        ymin_new = ymin - y1_crop
        xmax_new = xmax - x1_crop
        ymax_new = ymax - y1_crop
        if xmin_new < 0 or ymin_new < 0 or xmax_new > new_image_size[0] or ymax_new > new_image_size[1]:
            root.remove(obj)
            continue
        bbox.find('xmin').text = str(max(0, xmin_new))
        bbox.find('ymin').text = str(max(0, ymin_new))
        bbox.find('xmax').text = str(min(new_image_size[0], xmax_new))
        bbox.find('ymax').text = str(min(new_image_size[1], ymax_new))
    new_xml_name = os.path.join(save_path, f"{os.path.splitext(new_image_name)[0]}.xml")
    tree.write(new_xml_name)

def crop_images_with_xml(image_dir, xml_dir, save_dir, add_='a'):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    cropped_xml_save_dir = os.path.join(save_dir, "Annotations")
    if not os.path.exists(cropped_xml_save_dir):
        os.makedirs(cropped_xml_save_dir)
    cropped_image_save_dir = os.path.join(save_dir, "JPEGImages")
    if not os.path.exists(cropped_image_save_dir):
        os.makedirs(cropped_image_save_dir)
    for xml_file in tqdm(os.listdir(xml_dir)):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(xml_dir, xml_file)
            image_file = xml_file.replace('.xml', '.jpg')
            image_path = os.path.join(image_dir, image_file)
            if not os.path.exists(image_path):
                print(f"Image file {image_path} not found.")
                continue
            image = Image.open(image_path)
            w, h = image.size
            bounding_boxes = read_bbox(xml_path)
            if not bounding_boxes:
                print(f"No bounding boxes found in {xml_path}.")
                continue
            for idx, (xmin, ymin, xmax, ymax) in enumerate(bounding_boxes):
                # x1, x2, y1, y2 = get_squared_crop_box_random(xmin, xmax, ymin, ymax, w, h)
                x1, x2, y1, y2 = get_16w9h_crop_box_random(xmin, xmax, ymin, ymax, w, h)
                cropped_image = image.crop((x1, y1, x2, y2))
                cropped_image_name = f"{os.path.splitext(image_file)[0]}_{add_}_{idx}.jpg"
                cropped_image.save(os.path.join(cropped_image_save_dir, cropped_image_name))
                save_cropped_xml(xml_path, cropped_xml_save_dir, (x1, x1, y1, y1), (x2 - x1, y2 - y1), cropped_image_name)
                print(f"Cropped and saved {cropped_image_name} and corresponding XML.")

def crop_xml_od_dataset(image_dir, xml_dir, save_dir):
    crop_images_with_xml(image_dir, xml_dir, save_dir)

image_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\images"
xml_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1104\labels"
save_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1104_expand"
crop_images_with_xml(image_dir, xml_dir, save_dir)


  0%|          | 1/285 [00:00<01:13,  3.85it/s]

Cropped and saved 008c165c-df73-4dea-bb97-07d0bfece642_a_0.jpg and corresponding XML.
Cropped and saved 008c165c-df73-4dea-bb97-07d0bfece642_a_1.jpg and corresponding XML.
Cropped and saved 04d33559-a425-4ee0-8373-3d019eb82726_a_0.jpg and corresponding XML.
Cropped and saved 04d33559-a425-4ee0-8373-3d019eb82726_a_1.jpg and corresponding XML.
Cropped and saved 04d33559-a425-4ee0-8373-3d019eb82726_a_2.jpg and corresponding XML.


  1%|          | 2/285 [00:00<01:11,  3.96it/s]

Cropped and saved 04d33559-a425-4ee0-8373-3d019eb82726_a_3.jpg and corresponding XML.
Cropped and saved 04d33559-a425-4ee0-8373-3d019eb82726_a_4.jpg and corresponding XML.
Cropped and saved 04d33559-a425-4ee0-8373-3d019eb82726_a_5.jpg and corresponding XML.
Cropped and saved 0530c49f-668a-45aa-a3f2-dc6aed428498_a_0.jpg and corresponding XML.
Cropped and saved 0530c49f-668a-45aa-a3f2-dc6aed428498_a_1.jpg and corresponding XML.
Cropped and saved 0530c49f-668a-45aa-a3f2-dc6aed428498_a_2.jpg and corresponding XML.


  1%|          | 3/285 [00:00<01:26,  3.27it/s]

...

Cropped and saved fc23b5b8-4076-4208-973b-79c942b77fbf_a_3.jpg and corresponding XML.
Cropped and saved fc23b5b8-4076-4208-973b-79c942b77fbf_a_4.jpg and corresponding XML.
Cropped and saved fc23b5b8-4076-4208-973b-79c942b77fbf_a_5.jpg and corresponding XML.
Cropped and saved fd59a74b-11d5-4804-93ec-009df9642b3b_a_0.jpg and corresponding XML.


100%|█████████▉| 284/285 [01:17<00:00,  3.01it/s]

Cropped and saved fd59a74b-11d5-4804-93ec-009df9642b3b_a_1.jpg and corresponding XML.
Cropped and saved fd59a74b-11d5-4804-93ec-009df9642b3b_a_2.jpg and corresponding XML.
Cropped and saved fd59a74b-11d5-4804-93ec-009df9642b3b_a_3.jpg and corresponding XML.
Cropped and saved fd59a74b-11d5-4804-93ec-009df9642b3b_a_4.jpg and corresponding XML.
Cropped and saved fd59a74b-11d5-4804-93ec-009df9642b3b_a_5.jpg and corresponding XML.
Cropped and saved fd59a74b-11d5-4804-93ec-009df9642b3b_a_6.jpg and corresponding XML.


100%|██████████| 285/285 [01:17<00:00,  3.69it/s]

Cropped and saved ff433104-34d8-4c99-9d2d-5fec925005e2_a_0.jpg and corresponding XML.
Cropped and saved ff433104-34d8-4c99-9d2d-5fec925005e2_a_1.jpg and corresponding XML.
Cropped and saved ff433104-34d8-4c99-9d2d-5fec925005e2_a_2.jpg and corresponding XML.
Cropped and saved ff433104-34d8-4c99-9d2d-5fec925005e2_a_3.jpg and corresponding XML.
Cropped and saved ff433104-34d8-4c99-9d2d-5fec925005e2_a_4.jpg and corresponding XML.


In [8]:
# del empty data
def del_empty_xml_file_and_image(image_dir, xml_dir):
    """
    删除 xml_dir 中没有任何 bbox 的 XML 文件，以及对应的图像文件。

    参数：
        image_dir (str): 存放图像文件（JPG）的目录路径。
        xml_dir (str): 存放 XML 标注文件的目录路径。
    """
    deleted_xml_files = []
    deleted_image_files = []
    total_xml_files = 0

    for xml_file in os.listdir(xml_dir):
        if xml_file.endswith('.xml'):
            total_xml_files += 1
            xml_path = os.path.join(xml_dir, xml_file)
            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
                # 查找所有的 object 元素
                objects = root.findall('object')
                if not objects:
                    # 如果没有找到 object 元素，删除该 XML 文件和对应的图像文件
                    os.remove(xml_path)
                    deleted_xml_files.append(xml_file)
                    print(f"已删除空的 XML 文件：{xml_file}")

                    # 对应的图像文件名
                    base_name = os.path.splitext(xml_file)[0]
                    # 尝试匹配不同扩展名的图像文件
                    image_extensions = ['.jpg', '.jpeg', '.png']
                    image_deleted = False
                    for ext in image_extensions:
                        image_file = base_name + ext
                        image_path = os.path.join(image_dir, image_file)
                        if os.path.exists(image_path):
                            os.remove(image_path)
                            deleted_image_files.append(image_file)
                            print(f"已删除对应的图像文件：{image_file}")
                            image_deleted = True
                            break
                    if not image_deleted:
                        print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")
            except ET.ParseError:
                # 如果 XML 文件解析错误，也删除文件和对应的图像文件
                os.remove(xml_path)
                deleted_xml_files.append(xml_file)
                print(f"已删除无法解析的 XML 文件：{xml_file}")

                # 同样删除对应的图像文件
                base_name = os.path.splitext(xml_file)[0]
                image_extensions = ['.jpg', '.jpeg', '.png']
                image_deleted = False
                for ext in image_extensions:
                    image_file = base_name + ext
                    image_path = os.path.join(image_dir, image_file)
                    if os.path.exists(image_path):
                        os.remove(image_path)
                        deleted_image_files.append(image_file)
                        print(f"已删除对应的图像文件：{image_file}")
                        image_deleted = True
                        break
                if not image_deleted:
                    print(f"未找到对应的图像文件：{base_name}.[jpg|jpeg|png]")

    print(f"总共扫描了 {total_xml_files} 个 XML 文件，删除了 {len(deleted_xml_files)} 个空的 XML 文件。")
    print(f"删除了 {len(deleted_image_files)} 个对应的图像文件。")

img_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104_expand\JPEGImages"
xml_d = r"D:\ddesktop\monitoring\datadata\belthelmet\1104_expand\Annotations"
del_empty_xml_file_and_image(img_d, xml_d)


总共扫描了 1560 个 XML 文件，删除了 0 个空的 XML 文件。
删除了 0 个对应的图像文件。


In [12]:
import os
import shutil
import random

def sample_files(data_dir, target_dir, samples_=40):
    """
    从 data_dir 中随机抽取 samples_ 个文件，复制到 target_dir。

    参数：
        data_dir (str): 源文件夹路径。
        target_dir (str): 目标文件夹路径。
        samples_ (int): 要抽取的文件数量，默认为60。
    """
    # 如果目标目录不存在，则创建
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    # 获取 data_dir 中的所有文件列表
    files = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
    
    # 检查是否有足够的文件进行抽样
    if len(files) < samples_:
        print(f"文件数量不足，共有 {len(files)} 个文件，无法抽取 {samples_} 个样本。")
        return

    # 随机抽取 samples_ 个文件
    sampled_files = random.sample(files, samples_)

    # 复制文件到 target_dir
    for filename in sampled_files:
        src_path = os.path.join(data_dir, filename)
        dst_path = os.path.join(target_dir, filename)
        shutil.copy2(src_path, dst_path)  # 使用 copy2 保留文件元数据

    print(f"已成功复制 {samples_} 个文件到 {target_dir}。")


dir_ = r"D:\ddesktop\monitoring\datadata\for_anno\D04"
save_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1106_1309\images"

sample_files(dir_, save_dir)

已成功复制 40 个文件到 D:\ddesktop\monitoring\datadata\belthelmet\1106_1309\images。


In [14]:
def move_jpg_by_xml(xml_dir, jpg_dir, jpg_save_dir):
    name_list = [f.split('.xml')[0] for f in os.listdir(xml_dir) if f.endswith('.xml')]
    for fs in name_list:
        im_name = fs + '.jpg'
        im_path = os.path.join(jpg_dir, im_name)
        if os.path.isfile(im_path):
            im_path_save = os.path.join(jpg_save_dir, im_name)
            shutil.copy2(im_path, im_path_save) 

x_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1106_1309\labels"
im_dir = r"D:\ddesktop\monitoring\datadata\for_anno\nobelt"
im_save_dir = r"D:\ddesktop\monitoring\datadata\belthelmet\1106_1309\images"
move_jpg_by_xml(x_dir, im_dir, im_save_dir)
        

In [ ]:
def sample_od_dataset(dataset_root, target_root, _struct={"image": "images", "label": "labels"}, samples_=100):
    image_dir = os.path.join(dataset_root, _struct["image"])
    label_dir = os.path.join(dataset_root, _struct["label"])

    # 获取图像和标注的文件名（不含扩展名）
    im_namelist = [os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')]
    la_namelist = [os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.xml')]

    # 确保目标目录存在
    image_dir_save = os.path.join(target_root, _struct["image"])
    label_dir_save = os.path.join(target_root, _struct["label"])
    os.makedirs(image_dir_save, exist_ok=True)
    os.makedirs(label_dir_save, exist_ok=True)

    # 获取图像和标注文件的交集，确保图像和标注匹配
    namelist = [f for f in im_namelist if f in la_namelist]

    # 检查可用样本数是否足够
    if len(namelist) < samples_:
        print(f"可用的样本数只有 {len(namelist)}，少于请求的 {samples_} 个样本。将使用全部可用样本。")
        samples_ = len(namelist)

    # 随机打乱并抽取所需数量的样本
    random.shuffle(namelist)
    target_namelist = namelist[:samples_]

    for t in target_namelist:
        im_path = os.path.join(image_dir, t + '.jpg')
        im_path_save = os.path.join(image_dir_save, t + '.jpg')
        shutil.copy2(im_path, im_path_save)

        xml_path = os.path.join(label_dir, t + '.xml')
        xml_path_save = os.path.join(label_dir_save, t + '.xml')
        shutil.copy2(xml_path, xml_path_save)

    print(f"已成功复制 {samples_} 个样本到 {target_root}。")
    
